### Get first and last trial range (based on trained video) to plot


In [ ]:
import pandas as pd

# Define file paths
arhmm_latents_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_' + bodypart + '/706893/2024-05-28/dfs/arhmm_latents_df_.csv'
df_original_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_' + bodypart + '/706893/2024-05-28/dfs/df_original.csv'

# Load the CSV files
arhmm_latents_df = pd.read_csv(arhmm_latents_path)
df_original = pd.read_csv(df_original_path)

# Get the first Harp_Timestamp from arhmm_latents_df
first_harp_timestamp = arhmm_latents_df["Harp_Timestamp"].iloc[0]

# Find the closest next trial timestamp in df_original
first_analyzed_trial_idx = (df_original["start_time"] > first_harp_timestamp).idxmax()  # Get index of the first trial after first_harp_timestamp in analyzed video
first_analyzed_trial_event_timestamp = df_original.loc[first_analyzed_trial_idx, "start_time"]
first_analyzed_trial_number = df_original.loc[first_analyzed_trial_idx, "Trial"]

# Print results
print(f"First Harp_Timestamp from arhmm_latents_df: {first_harp_timestamp}")
print(f"Closest next trial Harp_Timestamp (start_time) from df_original: {first_analyzed_trial_event_timestamp} (Trial {first_analyzed_trial_number})")
print('')

# Get the last non-NaN Harp_Timestamp from arhmm_latents_df
last_harp_timestamp = arhmm_latents_df["Harp_Timestamp"].dropna().iloc[-1]

# # Find the closest prior trial timestamp in df_original
# Drop NaNs from the Harp_Timestamp column
df_original_clean = df_original.dropna(subset=["stop_time"])

# Find the index of the last trial before last_harp_timestamp
last_analyzed_trial_idx = df_original_clean[df_original_clean["stop_time"] < last_harp_timestamp].index[-1]
last_analyzed_trial_event_timestamp = df_original.loc[last_analyzed_trial_idx, "stop_time"]

# Get the trial number of the closest prior trial
last_analyzed_trial_number = df_original_clean.loc[last_analyzed_trial_idx, "Trial"]


# Print results
print(f"Last Harp_Timestamp from arhmm_latents_df: {last_harp_timestamp}")
print(f"Closest prior trial Harp_Timestamp from df_events: {last_analyzed_trial_event_timestamp} (Trial {last_analyzed_trial_number})")


In [ ]:
First Harp_Timestamp from arhmm_latents_df: 6574786.711488
Closest next trial Harp_Timestamp (start_time) from df_original: 6574787.602496 (Trial 9)

Last Harp_Timestamp from arhmm_latents_df: 6578185.45952
Closest prior trial Harp_Timestamp from df_events: 6578181.317504 (Trial 273)

### Get num_initial_frames_to_exclude and num_late_frames_to_exclude 

In [ ]:
import numpy as np
import pandas as pd

# Example usage
hdf5_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/data/xinxin/coupled-baiting_' + bodypart + '/706893/2024-05-28/data.hdf5'
set_harp_ts = True  

ae_latent_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_' + bodypart + '/706893/2024-05-28/dfs/ae_latents_df.csv'
ae_latents_df = pd.read_csv(ae_latent_path)

# Initialize exclusion counters
total_num_initial_frames_to_exclude, total_num_late_frames_to_exclude = 0, 0

if set_harp_ts:
    # Exclude frames from the beginning of the list
    selected_segments = np.arange(ae_latents_df['seg_index'].iloc[0])  

    n_frames_dict, total_num_initial_frames_to_exclude, total_num_frames_in_segments = get_frame_counts(hdf5_path, selected_segments)

    # Print results
    print("Total number of segments in the HDF5 file:", total_num_frames_in_segments)
    print("Total number of excluded frames for EARLY segments:", total_num_initial_frames_to_exclude)
    print("Frame counts per segments:", n_frames_dict)


    # Exclude frames from the end of the list
    selected_segments = np.arange(ae_latents_df['seg_index'].iloc[-1]+1, total_num_frames_in_segments)  

    n_frames_dict_, total_num_late_frames_to_exclude, _ = get_frame_counts(hdf5_path, selected_segments)

    # Print results
    print('')
    print("Total number of excluded frames for LATE segments:", total_num_late_frames_to_exclude)
    print("Frame counts per segments:", n_frames_dict_)


In [ ]:
Total number of segments in the HDF5 file: 178
Total number of excluded frames for EARLY segments: 4071
Frame counts per segments: {'trial_0000': 1000, 'trial_0001': 1071, 'trial_0002': 1000, 'trial_0003': 1000}

Total number of excluded frames for LATE segments: 3667
Frame counts per segments: {'trial_0174': 994, 'trial_0175': 953, 'trial_0176': 1000, 'trial_0177': 720}


### Get the number of frames in the selected_segments

In [ ]:
import h5py

def get_frame_counts(hdf5_path, selected_trials=None):
    """
    Reads an HDF5 file and extracts the number of frames for each selected trial.

    Parameters:
        hdf5_path (str): Path to the HDF5 file.
        selected_trials (list, optional): List of trial indices to include. If None, all trials will be used.

    Returns:
        dict: A dictionary with trial keys and their corresponding frame counts.
        int: The total number of frames across the selected trials.
        int: The total number of trials in the HDF5 file.
    """
    n_frames_dict = {}

    with h5py.File(hdf5_path, 'r') as f:
        images_group = f['images']
        total_num_trials = len(images_group.keys())  # Get the total number of trials

        # If no specific trials are selected, use all available trials
        if selected_trials is None:
            selected_trials = list(range(total_num_trials))

        # Loop over the selected trial indices
        for idx in selected_trials:
            trial_key = f"trial_{idx:04d}"
            if trial_key in images_group:
                n_frames = images_group[trial_key].shape[0]  # Extract number of frames
                n_frames_dict[trial_key] = n_frames

    # Compute total frames across selected trials
    total_num_frames = sum(n_frames_dict.values())

    return n_frames_dict, total_num_frames, total_num_trials



In [ ]:
total_num_frames_in_segments = 178
total_num_initial_frames_to_exclude = 4071
total_num_late_frames_to_exclude = 3667

In [ ]:
first_harp_timestamp = 6574786.711488
first_analyzed_trial_event_timestamp = 6574787.602496
first_analyzed_trial_number = 9

last_harp_timestamp = 6578185.480352
last_analyzed_trial_event_timestamp = 6578181.317504
last_analyzed_trial_number = 273

## Initiation

In [ ]:
import h5py

def get_frame_counts(hdf5_path, selected_trials=None):
    """
    Reads an HDF5 file and extracts the number of frames for each selected trial.

    Parameters:
        hdf5_path (str): Path to the HDF5 file.
        selected_trials (list, optional): List of trial indices to include. If None, all trials will be used.

    Returns:
        dict: A dictionary with trial keys and their corresponding frame counts.
        int: The total number of frames across the selected trials.
        int: The total number of trials in the HDF5 file.
    """
    n_frames_dict = {}

    with h5py.File(hdf5_path, 'r') as f:
        images_group = f['images']
        total_num_trials = len(images_group.keys())  # Get the total number of trials

        # If no specific trials are selected, use all available trials
        if selected_trials is None:
            selected_trials = list(range(total_num_trials))

        # Loop over the selected trial indices
        for idx in selected_trials:
            trial_key = f"trial_{idx:04d}"
            if trial_key in images_group:
                n_frames = images_group[trial_key].shape[0]  # Extract number of frames
                n_frames_dict[trial_key] = n_frames

    # Compute total frames across selected trials
    total_num_frames = sum(n_frames_dict.values())

    return n_frames_dict, total_num_frames, total_num_trials



In [ ]:
n_ae_latents_str = '08' #16
bodypart = 'ear'  # set body part accordingly, like 'nose', 'tongue_mouth'
fps_label = ''# '_480Hz'
bodypart = f"{bodypart}{fps_label}"


n_arhmm_states = [2,4,8,12,16,20,32]  
original_video_fps = 480


import os
# Define the directory path
dfs_dir = f"/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_{bodypart}/706893/2024-05-28/dfs"
# Ensure the directory exists
os.makedirs(dfs_dir, exist_ok=True)



nwb_dir = "/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/nwb"



## AE

In [ ]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from behavenet import get_user_dir, make_dir_if_not_exists
from behavenet.fitting.utils import get_expt_dir
from behavenet.fitting.utils import get_session_dir
from behavenet.fitting.utils import get_best_model_version
from behavenet.fitting.utils import get_lab_example

save_outputs = True  # true to save figures/movies to user's figure directory
format = 'png'  # figure format ('png' | 'jpeg' | 'pdf'); movies saved as mp4



# read the valuse of hparams from meta_tag.csv
import pandas as pd

# set the path to the meta_tag.csv file

# Path to your CSV file
# file_path = "meta_tag.csv"
meta_tag_file_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_' +bodypart+ '/706893/2024-05-28/ae/conv/' +n_ae_latents_str+ '_latents/' +bodypart+ '_3_views/version_0/meta_tags.csv'

# Load CSV
df_ = pd.read_csv(meta_tag_file_path)

# Extract frame rate
ae_model_experiment_name = df_[df_["key"] == "experiment_name"]["value"].values[0]
n_ae_latents = int(df_[df_["key"] == "n_ae_latents"]["value"].values[0])
experimenter_name = df_[df_["key"] == "lab"]["value"].values[0]
dataset_hparams_experiment_name = df_[df_["key"] == "expt"]["value"].values[0]
frame_rate = float(df_[df_["key"] == "frame_rate"]["value"].values[0])
ae_model_version = df_[df_["key"] == "version"]["value"].values[0]

print("ae_model_experiment_name:", ae_model_experiment_name)
print("n_ae_latents:", n_ae_latents)
print("experimenter_name:", experimenter_name)
print("dataset_hparams_experiment_name:", dataset_hparams_experiment_name)
print("frame_rate:", frame_rate)
print("ae_model_version:", ae_model_version)

# total_num_trials = total_num_trials # calculated before 





# Extract the version number
version = meta_tag_file_path.split("version_")[1].split("/")[0]

print("version assigned in meta_tag_file_path path: ", version) 
print("'version' as stored in meta_tag.csv: ", ae_model_version)

assert version == ae_model_version, "Version mismatch"





import pickle
import pandas as pd

# Define the updated file path
file_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_' +bodypart+ '/706893/2024-05-28/ae/conv/' +n_ae_latents_str+ '_latents/' +bodypart+ '_3_views/version_0/xinxin_coupled-baiting_' +bodypart+ '_706893_2024-05-28_latents.pkl'

# Load the updated pickle file
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Extract latents and trials from the updated file
ae_latents = data.get("latents", [])
ae_trials = data.get("trials", {})

# Convert latents to a structured DataFrame
ae_latent_trial_df = []
for i, arr in enumerate(ae_latents):
    if arr.size > 0:  # Ignore empty arrays
        df_temp = pd.DataFrame(arr)
        df_temp['seg_index'] = i  # Identify the latent group
        ae_latent_trial_df.append(df_temp)

proto_ae_latents_df = pd.concat(ae_latent_trial_df, ignore_index=True) if ae_latent_trial_df else pd.DataFrame()
# save proto_ae_latents_df to csv in dfs folder
proto_ae_latents_df.to_csv(f'/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_{bodypart}/706893/2024-05-28/dfs/proto_ae_latents_df.csv', index=False)


# Convert trials into a structured DataFrame
trial_type_df = pd.DataFrame([(k, list(v)) for k, v in ae_trials.items()], columns=["Trial Type", "Indices"])

# Display the first few rows of both DataFrames
print("=== Latents Data ===")
print(trial_type_df[1000:1300])  # Show first 10 rows

print("\n=== Trials Data ===")
print(trial_type_df)  # Show structured trial indices




train_indices = ae_trials.get("train", [])
test_indices = ae_trials.get("test", [])
val_indices = ae_trials.get("val", [])



print(len(train_indices), len(test_indices), len(val_indices))
print('train_indices:', train_indices)
print('test_indices:', test_indices)
print('val_indices:', val_indices)
# buffer makes sense now considering the way they assign data to data dtypes



# Combine all indices into one array
all_indices = np.concatenate((train_indices, test_indices, val_indices))

# Find the maximum value among all indices
max_value = np.max(all_indices)

# Create a full set of numbers from 0 to max_value (inclusive)
full_range = np.arange(max_value)

# Find missing numbers by computing the set difference
excluded_trials = np.setdiff1d(full_range, all_indices)

# print("Total num segmentss is ", total_num_trials)
print("Max trial index present is ", max_value)
print("Missing numbers from 0 to", max_value-1, "are:", excluded_trials)

# ???? has problem: excluded trial from the tail are not here



import numpy as np
train_indices = ae_trials.get("train", [])
test_indices = ae_trials.get("test", [])
val_indices = ae_trials.get("val", [])

# Combine all indices into one array and sort them
all_indices = np.concatenate((train_indices, test_indices, val_indices))
all_indices = np.sort(all_indices)

# get the max and min segment in the data
max_seg_idx = np.max(all_indices)
min_seg_idx = np.min(all_indices)

print("Max segment index present is ", max_seg_idx)
print("Min segment present is ", min_seg_idx)

# Find missing numbers by computing the set difference
excluded_trials = np.setdiff1d(np.arange(max_seg_idx), all_indices)
print("Missing numbers from 0 to", max_seg_idx, "segment are:", excluded_trials)


import numpy as np
import pandas as pd

# Example usage
hdf5_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/data/xinxin/coupled-baiting_' + bodypart + '/706893/2024-05-28/data.hdf5'
# set_harp_ts = True
# Initialize exclusion counters
total_num_initial_frames_to_exclude, total_num_late_frames_to_exclude = 0, 0

# Exclude frames from the beginning of the list
selected_segments = np.arange(min_seg_idx)

n_frames_dict, total_num_initial_frames_to_exclude, total_num_frames_in_segments = get_frame_counts(hdf5_path, selected_segments)

# Print results
print("Total number of segments in the HDF5 file:", total_num_frames_in_segments)
print("Total number of excluded frames for EARLY segments:", total_num_initial_frames_to_exclude)
print("Frame counts per segments:", n_frames_dict)


# Exclude frames from the end of the list
selected_segments = np.arange(max_seg_idx+1, total_num_frames_in_segments)  

n_frames_dict_, total_num_late_frames_to_exclude, _ = get_frame_counts(hdf5_path, selected_segments)

# Print results
print('')
print("Total number of excluded frames for LATE segments:", total_num_late_frames_to_exclude)
print("Frame counts per segments:", n_frames_dict_)

## ARHMM

In [ ]:
import os
import pickle
import scipy.io as sio
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from behavenet import get_user_dir, make_dir_if_not_exists
from behavenet.data.utils import get_transforms_paths
from behavenet.data.utils import load_labels_like_latents
from behavenet.fitting.utils import get_expt_dir
from behavenet.fitting.utils import get_session_dir
from behavenet.fitting.utils import get_best_model_version
from behavenet.fitting.utils import get_lab_example
from behavenet.plotting.arhmm_utils import *

save_outputs = True  # true to save figures/movies to user's figure directory
format = 'png'  # figure format ('png' | 'jpeg' | 'pdf'); movies saved as mp4
# model_class = 'arhmm'  # 'arhmm' | 'arhmm-labels'


# read the valuse of hparams from meta_tag.csv
import pandas as pd

# Path to your CSV file
# file_path = "meta_tag.csv"
meta_tag_file_path = '/local1/video-analysis/scratch/706893_2024-05-28_15-15-38/results/xinxin/coupled-baiting_' +bodypart+ '/706893/2024-05-28/arhmm/' +n_ae_latents_str+ '_latents/02_states/stationary/gaussian/arhmm_' +bodypart+ '_3_views/version_0/meta_tags.csv'

# Load CSV
df_ = pd.read_csv(meta_tag_file_path)

# Extract frame rate
arhmm_model_experiment_name = df_[df_["key"] == "experiment_name"]["value"].values[0]
n_ae_latents = int(df_[df_["key"] == "n_ae_latents"]["value"].values[0])
experimenter_name = df_[df_["key"] == "lab"]["value"].values[0]
dataset_hparams_experiment_name = df_[df_["key"] == "expt"]["value"].values[0]
frame_rate = float(df_[df_["key"] == "frame_rate"]["value"].values[0])
noise_type = df_[df_["key"] == "noise_type"]["value"].values[0]
n_arhmm_lags = int(df_[df_["key"] == "n_arhmm_lags"]["value"].values[0])
ae_experiment_name = df_[df_["key"] == "ae_experiment_name"]["value"].values[0]
model_class = df_[df_["key"] == "model_class"]["value"].values[0]
arhmm_model_version = df_[df_["key"] == "version"]["value"].values[0]

print("arhmm_model_experiment_name:", arhmm_model_experiment_name)
print("n_ae_latents:", n_ae_latents)
print("experimenter_name:", experimenter_name)
print("dataset_hparams_experiment_name:", dataset_hparams_experiment_name)
print("frame_rate:", frame_rate)
print("noise_type:", noise_type)
print("n_arhmm_lags:", n_arhmm_lags)
print("ae_experiment_name:", ae_experiment_name)
print("model_class:", model_class)
print("arhmm_model_version:", arhmm_model_version)
print("")



device = df_[df_["key"] == "device"]["value"].values[0]
batch_load = df_[df_["key"] == "batch_load"]["value"].values[0]
rng_seed_model = int(df_[df_["key"] == "rng_seed_model"]["value"].values[0])
as_numpy = df_[df_["key"] == "as_numpy"]["value"].values[0]
rng_seed_data = int(df_[df_["key"] == "rng_seed_data"]["value"].values[0])
trial_splits = df_[df_["key"] == "trial_splits"]["value"].values[0]
train_frac = float(df_[df_["key"] == "train_frac"]["value"].values[0])


print("device:", device)
print("batch_load:", batch_load)
print("rng_seed_model:", rng_seed_model)
print("as_numpy:", as_numpy)
print("rng_seed_data:", rng_seed_data)
print("trial_splits:", trial_splits)
print("train_frac:", train_frac)





# Extract the version number
version = meta_tag_file_path.split("version_")[1].split("/")[0]

print("version assigned in meta_tag_file_path path: ", version) 
print("'best version' as stored in meta_tag.csv: ", arhmm_model_version)

assert version == arhmm_model_version, "Version mismatch"





# Set up common hparams
hparams = {
    'data_dir': get_user_dir('data'),
    'save_dir': get_user_dir('save'),
    'experiment_name': arhmm_model_experiment_name,
    'model_class': model_class,
    'model_type': None,
    'noise_type': noise_type,
    'n_arhmm_lags': n_arhmm_lags,
    'transitions': 'stationary',
    'ae_experiment_name': ae_experiment_name,
    'ae_model_class': 'ae',
    'ae_model_type': 'conv',
    'ae_version': 'best',
    'n_ae_latents': n_ae_latents,
    'device': device,
    'as_numpy': as_numpy,
    'batch_load': batch_load,
    'rng_seed_model': rng_seed_model,
    'rng_seed_data': rng_seed_data,
    'trial_splits': trial_splits,
    'train_frac': train_frac,
}
get_lab_example(hparams, experimenter_name, dataset_hparams_experiment_name)





In [ ]:
import pandas as pd

# Construct the path where the CSV was saved.
arhmm_latents_path = os.path.join(get_session_dir(hparams)[0], 'dfs', 'arhmm_latents_df.csv')
# Read the CSV file into a DataFrame.
arhmm_latents_df = pd.read_csv(arhmm_latents_path)

# Define file paths
df_original_path = os.path.join(get_session_dir(hparams)[0], 'dfs', 'df_original.csv')
# Load the CSV files
df_original = pd.read_csv(df_original_path)

# Get the first Harp_Timestamp from arhmm_latents_df
first_harp_timestamp = arhmm_latents_df["Harp_Timestamp"].iloc[0]

# Find the closest next trial timestamp in df_original
first_analyzed_trial_idx = (df_original["start_time"] > first_harp_timestamp).idxmax()  # Get index of the first trial after first_harp_timestamp in analyzed video
first_analyzed_trial_event_timestamp = df_original.loc[first_analyzed_trial_idx, "start_time"]
first_analyzed_trial_number = df_original.loc[first_analyzed_trial_idx, "Trial"]

# Print results
print(f"First Harp_Timestamp from arhmm_latents_df: {first_harp_timestamp}")
print(f"Closest next trial Harp_Timestamp (start_time) from df_original: {first_analyzed_trial_event_timestamp} (Trial {first_analyzed_trial_number})")
print('')

# Get the last non-NaN Harp_Timestamp from arhmm_latents_df
last_harp_timestamp = arhmm_latents_df["Harp_Timestamp"].dropna().iloc[-1]

# # Find the closest prior trial timestamp in df_original
# Drop NaNs from the Harp_Timestamp column
df_original_clean = df_original.dropna(subset=["stop_time"])

# Find the index of the last trial before last_harp_timestamp
last_analyzed_trial_idx = df_original_clean[df_original_clean["stop_time"] < last_harp_timestamp].index[-1]
last_analyzed_trial_event_timestamp = df_original.loc[last_analyzed_trial_idx, "stop_time"]

# Get the trial number of the closest prior trial
last_analyzed_trial_number = df_original_clean.loc[last_analyzed_trial_idx, "Trial"]


# Print results
print(f"Last Harp_Timestamp from arhmm_latents_df: {last_harp_timestamp}")
print(f"Closest prior trial Harp_Timestamp from df_events: {last_analyzed_trial_event_timestamp} (Trial {last_analyzed_trial_number})")


# HMM

In [ ]:

import ssm
from ssm.util import find_permutation
from ssm.plots import gradient_cmap, white_to_color_cmap

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

color_names = [
    "windows blue",
    "red",
    "amber",
    "faded green",
    "dusty purple",
    "orange",
    "clay",
    "pink",
    "greyish",
    "mint",
    "light cyan",
    "steel blue",
    "forest green",
    "pastel purple",
    "salmon",
    "dark brown",
    "lavender",
    "tan",
    "maroon",
    "turquoise",
    "bright green",
    "gold",
    "sky blue"
    ]

colors_hmm = sns.xkcd_palette(color_names)  #????? bad naming
patch_cmap = gradient_cmap(colors_hmm)

# Speficy whether or not to save figures
save_figures = True

### Plot Reaction Time vs Trial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# === Load the data ===
df_original = pd.read_csv(dfs_dir + '/df_original.csv')

# === Set your analysis parameters ===
# first_analyzed_trial_number = 1  # Replace this with your actual desired trial number
# Parameters
excluding_initial_late = True  # Set to False to include all data

# Filter to rewarded trials only
df_original_cp = df_original.copy()

# === Step 1: Compute reaction time ===
df_original_cp['reaction_time'] = df_original_cp['reward_outcome_time'] - df_original_cp['goCue_start_time']

# === Step 2: Filter out trials before the starting point ===
# df_filtered = df_original_cp[df_original_cp['Trial'] >= first_analyzed_trial_number].copy()
# Apply time filtering if enabled
if excluding_initial_late:
    df_original_cp = df_original_cp[
        (df_original_cp['Trial'] >= first_analyzed_trial_number) &
        (df_original_cp['Trial'] <= last_analyzed_trial_number)
    ]

# === Step 3: Drop rows with missing reaction time ===
df_original_cp = df_original_cp.dropna(subset=['reaction_time', 'Trial'])

# === Step 4: Plot RT vs. Trial ===
plt.figure(figsize=(16, 3))
plt.scatter(df_original_cp['Trial'], df_original_cp['reaction_time'], color='blue', alpha=0.7)

plt.xlabel('Trial Number')
plt.ylabel('Reaction Time (s)')
plt.title(f'Reaction Time vs. Trial (starting from Trial {first_analyzed_trial_number})')
plt.grid(True)
plt.tight_layout()
plt.show()


temp

In [ ]:
import cv2
import os

# === Parameters ===
selected_cluster = 1  # Change this to target the cluster of interest
output_dir = f"./cluster_{selected_cluster}_videos"
# total_num_initial_frames_to_exclude = 1000  # Adjust to your actual value
# original_video_path = '/local1/video-analysis/videos/706893_2024-05-28_15-15-38/ear/left_ear.mp4'  # Set to your original video file path
original_video_path = '/local1/video-analysis/videos/706893_2024-05-28_15-15-38/face_left/face_left_48fps.mp4'


# Create output directory
os.makedirs(output_dir, exist_ok=True)

# === Get Trials from the Selected Cluster ===
selected_trials = clustered_df[clustered_df['Cluster'] == selected_cluster]['Trial'].unique()

# === Load the video ===
cap = cv2.VideoCapture(original_video_path)
if not cap.isOpened():
    raise IOError("Error opening video file")

# === Loop through each trial ===
for trial_num in selected_trials:
    if trial_num not in trial_data:
        print(f"Trial {trial_num} not in trial_data; skipping.")
        continue
    
    df_trial = trial_data[trial_num]
    frame_indices = df_trial['Timestamp'].astype(int).tolist()
    corrected_indices = [i + total_num_initial_frames_to_exclude for i in frame_indices]
    
    # Set up video writer
    output_path = os.path.join(output_dir, f"cluster{selected_cluster}_trial{trial_num}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    cap.read()  # Dummy read to initiate

    # Get frame properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Read and write selected frames
    for frame_idx in corrected_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            writer.write(frame)
        else:
            print(f"Warning: Could not read frame {frame_idx} for Trial {trial_num}")
    
    writer.release()
    print(f"Saved: {output_path}")

print(fps)
# Release the video object
cap.release()
print("All trial clips exported.")
